In [12]:
using LinearAlgebra, Random

# Definición correcta que acepta ambos keywords: tol y full
function svd_eig(A_in; tol=1e-12, full=false)
    A = Array{Float64}(A_in)
    m, n = size(A)

    AtA = A' * A
    ev = eigen(AtA)
    vals = ev.values
    vecs = ev.vectors

    idx = sortperm(vals, rev=true)
    vals_sorted = vals[idx]
    Vfull = vecs[:, idx]

    s_all = sqrt.(clamp.(vals_sorted, 0.0, Inf))
    r = count(x -> x > tol, s_all)

    if r == 0
        U_comp = zeros(Float64, m, 0)
        s_comp = Float64[]
        V_comp = zeros(Float64, n, 0)
    else
        V_comp = Vfull[:, 1:r]
        s_comp = s_all[1:r]
        U_comp = (A * V_comp) ./ reshape(s_comp, 1, r)
    end

    if !full
        return U_comp, s_comp, V_comp
    end

    # construcción SVD completa
    # preparar Ufull
    if r == 0
        Ufull = qr!(copy(randn(m, m))).Q
    else
        Ufull = zeros(Float64, m, m)
        Ufull[:, 1:r] = U_comp
        function complete_orthonormal!(U::Matrix{Float64}, start_col::Int)
            m = size(U, 1)
            current = start_col - 1
            Qrand = qr!(copy(randn(m, m))).Q
            for j in 1:size(Qrand, 2)
                if current >= m
                    break
                end
                q = Qrand[:, j]
                if current > 0
                    proj = U[:, 1:current] * (U[:, 1:current]' * q)
                    q -= proj
                end
                nq = norm(q)
                if nq > 1e-12
                    current += 1
                    U[:, current] = q / nq
                end
            end
            if current < m
                Q2 = qr!(copy(randn(m, m))).Q
                col = current + 1
                for j in 1:size(Q2, 2)
                    if col > m
                        break
                    end
                    U[:, col] = Q2[:, j]
                    col += 1
                end
            end
            return nothing
        end

        if r < m
            complete_orthonormal!(Ufull, r+1)
        end
    end

    Σ = zeros(Float64, m, n)
    for i in 1:min(m, n)
        Σ[i,i] = s_all[i]
    end

    return Ufull, Σ, Vfull
end


svd_eig (generic function with 2 methods)

In [ ]:
A = [3.0 1.0 1.0;
     -1.0 3.0 1.0;
      0.0 2.0 4.0;
      1.0 0.0 2.0]   # 4x3

# SVD compacta (económica)
Uc, sc, Vc = svd_eig(A)               # equivale a svd_eig(A; full=false)

# SVD completa
Ufull, Σfull, Vfull = svd_eig(A; full=true)
